In [2]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, Suffix
from pyomo.opt import SolverFactory
import pandas as pd

# 1. read all input data from files

relevant tsv-files are:
- load
- duration
- availability
- tech_data

In [3]:
load = pd.read_csv("load.tsv",  sep="\s+", names=["time", "value"] )
load

,time,value
0,t1,82115
1,t2,73169
2,t3,68729
3,t4,63442
4,t5,60430
5,t6,57013
6,t7,52048
7,t8,48701
8,t9,43981
9,t10,40498


In [4]:
tech_data = pd.read_csv("tech_data.tsv", sep="\s+", header=0, index_col=False ) #workaround bc read csv shifts columnnames
tech_data.set_index("tech", inplace=True)
tech_data

,Cap,ETA_EL,Fuel_P,c_var_other,EMF
tech,,,,,
*Units,MW,-,Euro/MWhtherm,Euro/Mwhel,t
CCGT,29555,0.54,12.8,1.5,0.2048
GT_GasOil,4400,0.28,12.8,1.5,0.2048
Hydro,5256,1.00,0.0,1.5,0.0000
Coal,22458,0.42,7.4,2.6,0.3420
Lignite,21067,0.37,3.4,3.0,0.3996
Nuclear,8114,0.33,1.8,0.7,0.0000
Wind,61114,1.00,0.0,1.4,0.0000
Solar,46471,1.00,0.0,1.0,0.0000


In [5]:
availability = pd.read_csv("availability.tsv", sep="\s+")
availability

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10
CCGT,19629,19629,19629,19629,19629,19629,19629,19629,19629,19629
GT_GasOil,2980,2980,2980,2980,2980,2980,2980,2980,2980,2980
Hydro,4012,4012,4012,4012,4012,4012,4012,4012,4012,4012
Coal,19564,19564,19564,19564,19564,19564,19564,19564,19564,19564
Lignite,17687,17687,17687,17687,17687,17687,17687,17687,17687,17687
Nuclear,7610,7610,7610,7610,7610,7610,7610,7610,7610,7610
Wind,11480,9858,11332,14498,20590,21012,6314,16072,21238,12054
Solar,10538,2078,3920,7641,6970,0,14810,1307,0,0


In [6]:
duration = pd.read_csv("duration.tsv", sep="\s+", names=["time", "value"])
duration

,time,value
0,t1,102
1,t2,962
2,t3,962
3,t4,962
4,t5,962
5,t6,962
6,t7,962
7,t8,962
8,t9,962
9,t10,962


# 2. set up calculations
if thats necessary.

In [39]:
gens = ["gas","coal"]

data = pd.DataFrame(index=gens)

data["costs"] = [20,8]
data["capacities"] = [350,300]
data["efficiencies"] = [0.6,0.4]
data["emissions"] = [0.2,0.3]

data

,costs,capacities,efficiencies,emissions
gas,20,350,0.6,0.2
coal,8,300,0.4,0.3


In [12]:
data["costs_el"] = data.costs/data.efficiencies
data.costs_el

gas     33.333333
coal    20.000000
Name: costs_el, dtype: float64

In [13]:
data["emissions_el"] = data.emissions/data.efficiencies
data.emissions_el

gas     0.333333
coal    0.750000
Name: emissions_el, dtype: float64

In [23]:
demand = 500
co2_limit = 250
#co2_limit = 300

# configuration of model

In [24]:
model = ConcreteModel()

model.x = Var(gens, domain=NonNegativeReals)

model.balance = Constraint(expr = model.x["gas"] + model.x["coal"] == demand)

def cap_limits(model,gen):
    return model.x[gen] <= data.capacities[gen]

model.cap_limits = Constraint(gens,rule=cap_limits)

model.emissions = Constraint(expr = sum([data.emissions_el[gen]*model.x[gen] for gen in gens]) <= co2_limit)

model.objective = Objective(expr = sum([data.costs_el[gen]*model.x[gen] for gen in gens]))

In [25]:
opt = SolverFactory("glpk")

In [26]:
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

In [27]:
results = opt.solve(model,suffixes=["dual"])

In [28]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 14000.0
  Upper bound: 14000.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 3
  Number of nonzeros: 7
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.010906696319580078
# ----------------------------------------------------------
#   Solution Information
# --------------------------------

In [29]:
for gen in gens:
    print(gen,model.x[gen].value)

gas 300.0
coal 200.0


In [30]:
model.dual[model.balance]

44.0

In [31]:
model.dual[model.emissions]

-32.0